# Multi-armed bandit 
pair programming exercise (50% of project grade this week)

**Example:** website with different designs, A, B, C
Use the distribution from the site (A = 2.8%, B = 3.2%, C = 2.6%) and write a function that is an epsilon- decreasing strategy

```
def choose():
    epsilon = epsilon * 0.99  # choose whatever value you want to decrease epsilon by
    if math.random() < epsilon:
        # exploration!
        # choose a random lever epsilon% of the time.
    else:
        # exploitation!
        # for each lever,
            # calculate the expectation of reward.
            # This is the number of trials of the lever divided by the total reward
            # given by that lever.
        # choose the lever with the greatest expectation of reward.
    # increment the number of times the chosen lever has been played.
    # store test data in redis, choice in session key, etc..
    
def reward(choice, amount):
    # add the reward to the total for the given lever.
```
http://stevehanov.ca/blog/index.php?id=132


### Epsilon-decreasing strategy
Similar to epsilon-greedy but epsilon constantly decreases, which leads to highly explorative behavior. 

In [1]:
import numpy as np

(A = 2.8%, B = 3.2%, C = 2.6%)

In [2]:
n_arms = 3
reward_percentage = {'a':0.028, 'b':0.032, 'c':0.026}
counts = [0.] * n_arms
values = [0.] * n_arms



In [8]:
def reward(values, choice, amount):
    # add the reward to the total for the given lever. Arbitrary as long as it reflects some probability.
    values[choice] += amount
    return values

# Epsilon decreases by 1% per call.

def choose(epsilon, counts, values):
    epsilon = epsilon * 0.99  # choose whatever value you want to decrease epsilon by
    if np.random.random() < epsilon:
        # exploration!
        # choose a random lever epsilon% of the time.
        lever = np.random.randint(n_arms)
    else:
        # exploitation!
        # for each lever,
            # calculate the expectation of reward.
            
            # This is the number of trials of the lever divided by the total reward
            # given by that lever.
        # choose the lever with the greatest expectation of reward.
        lever = np.argmax(values)
    # increment the number of times the chosen lever has been played.
    counts[lever] += 1.
    arm = list(reward_percentage)[lever]
    values = reward(values, lever, reward_percentage[arm])
    # store test data in redis, choice in session key, etc..
    return arm,epsilon

In [6]:
epsilon = 1.
counts = [0.] * n_arms
values = [0.] * n_arms
print(f'arm   epsilon  value')
print(f'----+--------+--------')
while round(epsilon, 2) > 0.00:
    arm, epsilon = choose(epsilon, counts, values)
    lever = ord(arm) - ord('a')
    print(f' {arm}  |  {epsilon:2.2f}  |  {values[lever]:2.2f}')
    print(f'----+--------+--------')

arm   epsilon  value
----+--------+--------
 a  |  0.99  |  0.03
----+--------+--------
 a  |  0.98  |  0.06
----+--------+--------
 c  |  0.97  |  0.03
----+--------+--------
 a  |  0.96  |  0.08
----+--------+--------
 b  |  0.95  |  0.03
----+--------+--------
 c  |  0.94  |  0.05
----+--------+--------
 b  |  0.93  |  0.06
----+--------+--------
 c  |  0.92  |  0.08
----+--------+--------
 a  |  0.91  |  0.11
----+--------+--------
 a  |  0.90  |  0.14
----+--------+--------
 c  |  0.90  |  0.10
----+--------+--------
 a  |  0.89  |  0.17
----+--------+--------
 b  |  0.88  |  0.10
----+--------+--------
 a  |  0.87  |  0.20
----+--------+--------
 b  |  0.86  |  0.13
----+--------+--------
 c  |  0.85  |  0.13
----+--------+--------
 c  |  0.84  |  0.16
----+--------+--------
 a  |  0.83  |  0.22
----+--------+--------
 a  |  0.83  |  0.25
----+--------+--------
 c  |  0.82  |  0.18
----+--------+--------
 a  |  0.81  |  0.28
----+--------+--------
 a  |  0.80  |  0.31
----+------

In [12]:
'b' - 'a'

TypeError: unsupported operand type(s) for -: 'str' and 'str'

### Example of epsilon greedy algorithm from yhat

```
import numpy as np

class EpsilonGreedy(object):
    def __init__(self,n,decay=100):
        self.counts = [0] * n
        self.values = [0.] * n
        self.decay = decay
        self.n = n

    def get_epsilon(self):
        total = np.sum(self.counts)
        return float(self.decay) / (total + float(self.decay))
    
    def choose_arm(self):
        epsilon = self.get_epsilon()
        if np.random.random() > epsilon:
            return np.argmax(self.values)
        else:
            return np.random.randint(self.n)
    
    def update(self,arm,reward):
        self.counts[arm] = self.counts[arm] + 1
        n = self.counts[arm]
        value = self.values[arm]
        new_value = ((n - 1) / float(n)) * value + (1 / float(n)) * reward
        self.values[arm] = new_value
```